In [1]:
from sigvisa.treegp.bcm.multi_shared_bcm import MultiSharedBCM, Blocker, sample_synthetic
from sigvisa.treegp.bcm.local_regression import BCM
from sigvisa.treegp.bcm.bcmopt import OutOfTimeError


from sigvisa.treegp.gp import GPCov, GP, mcov, prior_sample, dgaussian
from sigvisa.treegp.util import mkdir_p
import numpy as np
import scipy.stats
import scipy.optimize
import time
import os
import sys
import cPickle as pickle

In [2]:
import scipy.io

def load_sarcos():
    train_fname = "/home/dmoore/sarcos/sarcos_inv.mat"
    test_fname = "/home/dmoore/sarcos/sarcos_inv_test.mat"
    train_X = scipy.io.loadmat(train_fname)['sarcos_inv']
    train_y = train_X[:, 21]
    train_X = train_X[:, :21]
    return train_X, train_y

tX, ty = load_sarcos()
ty -= np.mean(ty)
ty /= np.std(ty)
print tX.shape, ty.shape

(44484, 21) (44484,)


In [3]:

#from sklearn.cluster import KMeans
#km = KMeans(n_clusters=80, init='k-means++', n_init=10, max_iter=300, tol=0.0001, precompute_distances='auto', verbose=1, random_state=None, copy_x=True, n_jobs=1)

In [4]:
def cluster_rpc((X, y), target_size):
    n = X.shape[0]
    if n < target_size:
        return [(X, y),]
    
    x1 = X[np.random.randint(n), :]
    x2 = x1
    while (x2==x1).all():
        x2 = X[np.random.randint(n), :]
    
    # what's the projection of x3 onto (x1-x2)? 
    # imagine that x2 is the origin, so it's just x3 onto x1. 
    # This is x1 * <x3, x1>/||x1||
    cx1 = x1 - x2
    nx1 = cx1 / np.linalg.norm(cx1)
    alphas = [ np.dot(xi-x2, nx1)  for xi in X]
    median = np.median(alphas)
    C1 = (X[alphas < median], y[alphas < median])
    C2 = (X[alphas >= median], y[alphas >= median])
    
    L1 = cluster_rpc(C1, target_size=target_size)
    L2 = cluster_rpc(C2, target_size=target_size)
    return L1 + L2
    
    
CC = cluster_rpc((tX, ty), target_size=256)

In [5]:
def sort_by_cluster(clusters):
    Xs, ys = zip(*clusters)
    SX = np.vstack(Xs)
    SY = np.concatenate(ys).reshape((-1, 1))
    block_boundaries = []
    n = 0
    for (X,y) in clusters:
        cn = X.shape[0]
        block_boundaries.append((n, n+cn))
        n += cn
    return SX, SY, block_boundaries

SX, SY, block_boundaries = sort_by_cluster(CC)

In [9]:

nv = 0.1
cov = GPCov(wfn_params=[1.0], dfn_params=np.ones((1, SX.shape[1])), dfn_str="euclidean", wfn_str="se")


mbcm = MultiSharedBCM(SX, SY, block_boundaries, cov, nv, 
                      dy=1, neighbor_threshold=0.1, 
                      nonstationary=False, 
                      nonstationary_prec=False)

0: neighbors count 0
10: neighbors count 4
20: neighbors count 1
30: neighbors count 1
40: neighbors count 3


KeyboardInterrupt: 

In [10]:
print len(mbcm.neighbors)

2381


In [87]:
C0 = np.ones((1, 23))
C0[0,0] = 0.1
c0 = C0.flatten()

def cov_prior(c):
    mean = -1
    std = 3
    r = (c-mean)/std
    ll = -.5*np.sum( r**2)- .5 *len(c) * np.log(2*np.pi*std**2)
    lderiv = -(c-mean)/(std**2)
    return ll, lderiv

sstep = [0,]
maxsec=3600
t0 = time.time()
def lgpllgrad(c):

    FC = np.exp(c.reshape(C0.shape))
    mbcm.update_covs(FC)
    #np.save(os.path.join(d, "step_%05d_cov.npy" % sstep[0]), FC)

    ll, gX, gC = mbcm.llgrad(local=True, grad_X=False, 
                             grad_cov=True,
                                   parallel=False)


    prior_ll, prior_grad = cov_prior(c)
    ll += prior_ll
    gC = (gC * FC).flatten() + prior_grad

    print "%d %.2f %.2f" % (sstep[0], time.time()-t0, ll)
    print FC
    
    sstep[0] += 1

    if time.time()-t0 > maxsec:
        raise OutOfTimeError

    return -ll, -gC

bounds = [(-10, 5)]*len(c0)
r = scipy.optimize.minimize(lgpllgrad, c0, jac=True, method="l-bfgs-b", bounds=bounds)


0 38.02 -67822.18
[[ 1.10517092  2.71828183  2.71828183  2.71828183  2.71828183  2.71828183
   2.71828183  2.71828183  2.71828183  2.71828183  2.71828183  2.71828183
   2.71828183  2.71828183  2.71828183  2.71828183  2.71828183  2.71828183
   2.71828183  2.71828183  2.71828183  2.71828183  2.71828183]]
1 76.22 -143439992.31
[[  4.53999298e-05   4.53999298e-05   1.48413159e+02   1.48413159e+02
    1.48413159e+02   1.48413159e+02   1.48413159e+02   1.48413159e+02
    1.48413159e+02   1.48413159e+02   1.48413159e+02   1.48413159e+02
    1.48413159e+02   1.48413159e+02   1.48413159e+02   1.48413159e+02
    1.48413159e+02   1.48413159e+02   1.48413159e+02   1.48413159e+02
    1.48413159e+02   1.48413159e+02   1.48413159e+02]]
2 114.28 -26122.86
[[ 0.0543667   0.10224333  8.96091745  8.96091745  8.96091745  8.96091745
   8.96091745  8.96091745  8.96091745  8.96091745  8.96091745  8.96091745
   8.96091745  8.96091745  8.96091745  8.96091745  8.96091745  8.96091745
   8.96091745  8.96091745  8

In [90]:
r1 = r
print "LOCAL GP HPARAMS"
print r1.x

def prediction_error_bcm(self, X=None, cov=None, local_dist=1.0):
    ntest = self.n-self.ntrain
    yd = self.SY.shape[1]
    mbcm = self.build_mbcm(X=X, cov=cov, local_dist=local_dist)

    # TODO: predict with respect to local covs instead of a global test cov
    p = mbcm.train_predictor()
    PM, PC = p(self.Xtest, test_noise_var=self.noise_var)
    PP = np.linalg.inv(PC)
    PR = self.Ytest-PM

    ll =  -.5 * np.sum(PP *  np.dot(PR, PR.T))
    ll += -.5 * np.linalg.slogdet(PC)[1]
    ll += -.5 * ntest * yd * np.log(2*np.pi)
    return ll / (ntest * yd)

LOCAL GP HPARAMS
[-4.89352177  0.28214431  0.18352165  0.34138676  0.58716482  0.40162707
  0.56416346  1.69768825  0.24985834  1.10916372  1.95523951  1.90959258
  1.91216458  2.30070937  2.13424957  1.47005619  2.97595552  4.06292368
  3.78080634  3.77582648  4.21674619  3.76479572  3.50156498]


In [92]:

nv = 0.1
cov = GPCov(wfn_params=[1.0], dfn_params=np.ones((1, SX.shape[1])), dfn_str="euclidean", wfn_str="se")


mbcm = MultiSharedBCM(SX, SY, block_boundaries, cov, nv, 
                      dy=1, neighbor_threshold=1.0, 
                      nonstationary=False, 
                      nonstationary_prec=False)

In [93]:
print mbcm.neighbors

[(1, 0), (2, 0), (3, 2), (5, 4), (6, 0), (6, 1), (7, 5), (7, 6), (8, 0), (8, 2), (8, 3), (10, 2), (10, 3), (10, 8), (11, 2), (11, 8), (11, 10), (13, 3), (13, 10), (13, 12), (14, 2), (14, 3), (14, 8), (14, 10), (14, 11), (14, 12), (14, 13), (15, 3), (15, 8), (15, 10), (15, 11), (15, 12), (15, 13), (15, 14), (16, 8), (16, 11), (17, 16), (19, 8), (19, 10), (19, 11), (19, 15), (19, 16), (19, 18), (20, 3), (20, 8), (20, 10), (20, 11), (20, 15), (20, 19), (21, 8), (21, 10), (21, 11), (21, 15), (21, 16), (21, 19), (21, 20), (22, 0), (22, 8), (23, 3), (23, 8), (23, 10), (23, 11), (23, 16), (23, 20), (23, 21), (23, 22), (24, 12), (24, 13), (24, 15), (24, 16), (24, 17), (24, 19), (25, 11), (25, 12), (25, 13), (25, 15), (25, 16), (25, 19), (25, 21), (25, 24), (26, 8), (26, 10), (26, 11), (26, 12), (26, 13), (26, 14), (26, 15), (26, 16), (26, 17), (26, 19), (26, 21), (26, 24), (26, 25), (27, 11), (27, 12), (27, 13), (27, 15), (27, 16), (27, 19), (27, 21), (27, 23), (27, 24), (27, 25), (27, 26), (2

In [94]:
C0 = np.ones((1, 23))
C0[0,0] = 0.1
c0 = C0.flatten()

def cov_prior(c):
    mean = -1
    std = 3
    r = (c-mean)/std
    ll = -.5*np.sum( r**2)- .5 *len(c) * np.log(2*np.pi*std**2)
    lderiv = -(c-mean)/(std**2)
    return ll, lderiv

sstep = [0,]
maxsec=36000
t0 = time.time()
def lgpllgrad(c):

    FC = np.exp(c.reshape(C0.shape))
    mbcm.update_covs(FC)
    #np.save(os.path.join(d, "step_%05d_cov.npy" % sstep[0]), FC)

    ll, gX, gC = mbcm.llgrad(local=True, grad_X=False, 
                             grad_cov=True,
                                   parallel=False)


    prior_ll, prior_grad = cov_prior(c)
    ll += prior_ll
    gC = (gC * FC).flatten() + prior_grad

    print "%d %.2f %.2f" % (sstep[0], time.time()-t0, ll)
    print FC
    
    sstep[0] += 1

    if time.time()-t0 > maxsec:
        print "killing, final log covs", c.reshape(C0.shape)
        raise OutOfTimeError

    return -ll, -gC

bounds = [(-10, 5)]*len(c0)
r = scipy.optimize.minimize(lgpllgrad, c0, jac=True, method="l-bfgs-b", bounds=bounds)


0 499.93 -65108.06
[[ 1.10517092  2.71828183  2.71828183  2.71828183  2.71828183  2.71828183
   2.71828183  2.71828183  2.71828183  2.71828183  2.71828183  2.71828183
   2.71828183  2.71828183  2.71828183  2.71828183  2.71828183  2.71828183
   2.71828183  2.71828183  2.71828183  2.71828183  2.71828183]]
1 999.07 -192257298.32
[[  4.53999298e-05   4.53999298e-05   1.48413159e+02   1.48413159e+02
    1.48413159e+02   1.48413159e+02   1.48413159e+02   1.48413159e+02
    1.48413159e+02   1.48413159e+02   1.48413159e+02   1.48413159e+02
    1.48413159e+02   1.48413159e+02   1.48413159e+02   1.48413159e+02
    1.48413159e+02   1.48413159e+02   1.48413159e+02   1.48413159e+02
    1.48413159e+02   1.48413159e+02   1.48413159e+02]]
2 1498.18 3409.33
[[ 0.05214854  0.09770845  9.10997452  9.10997452  9.10997452  9.10997452
   9.10997452  9.10997452  9.10997452  9.10997452  9.10997452  9.10997452
   9.10997452  9.10997452  9.10997452  9.10997452  9.10997452  9.10997452
   9.10997452  9.10997452  